# Prédiction de la qualité d'un film à partir de ses caractéristiques

## Présentation du projet
Le but de notre projet est de créer un modèle économétrique pour prédire le succès d'un film à partir de ses caractéristiques, comme sa durée, son genre, son réalisateur, etc. 

Nous avons décidé d'utiliser les données d'IMDb, un site de notation et de référencement des films. L'avantage de cette plateforme est qu'elle permet aux internautes de noter les films qu'ils ont vu, ce qui sera ce que l'on considère comme la mesure du succès d'un film. Par ailleurs, de nombreuses informations qui nous seront utiles sont présentes sur ce site et sont plus directement accessibles que sur une plateforme comme Wikipedia.

## Commençons par la mise en place du jeu de données
Nous importons une base mise à disposition par IMDb, que nous rendons exploitable par quelques opérations élémentaires.
Nous ne conservons que les films qui ont 2.000 votes ou plus : cela permet d'une part d'éviter de considérer les films trop peu votés pour que leur note moyenne soit pertinente, et d'autre part d'avoir un jeu de données plus léger.

In [1]:
import pandas as pd
movies_rating = pd.read_csv("https://datasets.imdbws.com/title.ratings.tsv.gz")
movies_rating_clean = movies_rating["tconst\taverageRating\tnumVotes"].str.split("\\t", expand=True)
movies_rating_clean.columns = ['ID', 'Note_moyenne', 'Nombre_de_votes']
movies_rating_clean['Note_moyenne'] = movies_rating_clean['Note_moyenne'].astype(float)
movies_rating_clean['Nombre_de_votes'] = movies_rating_clean['Nombre_de_votes'].astype(float)
movies_rating_filtré = movies_rating_clean[movies_rating_clean.Nombre_de_votes > 1999]

**Nous voilà maintenant en possession d'une première base de travail**

Le problème de ce jeu de donné téléchargé, c'est qu'il ne contient que des informations sur les votes des films. Il n'y a aucune mention d'autres caractéristiques dont nous pourrons avoir besoin, comme son cast. Nous avons contacté les services d'IMDb, mais leur API est payant... Nous avons donc choisi de scraper les informations dont nous avons besoin.

Cela dit, la base téléchargée va nous être particulièrement utile ! Nous avons à disposition les identifiants de tous les films de la plateforme qui ont recueilli 2000 votes ou plus, et l'URL des pages des films s'écrit à partir de cet identifiant.

Après avoir essayé de scraper les éléments en cherchant des chemins d'accès manuellement dans le code HTML, nous avons trouvé dans chaque page un dictionnaire qui comprend les caractéristiques principales des films. Le code suivant permet de recueillir ces données, les traiter pour les rendre exploitables, et les insérer dans un dataframe.

Nous avions des ambitions assez importantes quant aux variables à retenir, mais certains éléments étaient intraçables dans la soupe que nous donne BeautifulSoup. Le dictionnaire que nous pouvons scraper ne contient malheureusement pas des données comme le budget, la langue d'origine, etc.

In [2]:
#On fabrique le squelette du dataframe que l'on va remplir au fur et à mesure du scrap ; on le fusionnera par la suite
#avec la database téléchargée plus tôt
#On indique le nom des colonnes, qui sont les variables que l'on choisit de conserver
#contentRating est la classification d'age, creator est la société de production
df = pd.DataFrame(columns=['name', 'alternateName', 'url', 'contentRating', 'datePublished', 'genre', 'actor', 'director', 'creator', 'Origine', 'Budget', 'duration', 'keywords'])

In [ ]:
from bs4 import BeautifulSoup as bs
import requests
from random import seed
import time
import json
import re


#C'est une liste des objets inutiles dans le scrap des pages ; je retire aussi le contenu du dataframe qu'on a déjà (les votes)
superflu = ["@context", "@type", "image", "description", "review", "trailer", "aggregateRating"]
#Celui-ci servira à retirer les images des scénaristes, etc
superflu2 = ['@type', 'url']

session_obj = requests.Session()


#On boucle sur chaque film qu'on considère
#Le compteur est cosmétique : il sert à nous rassurer sur le fait que tout se passe bien pendant le scrap
compteur = 0
for ID in movies_rating_filtré['ID'] :
  compteur = compteur+1
  print(compteur)
  try: #On utilise un try except au cas où on aurait un problème sur une page : on ne veut pas que l'exécution s'arrête après des heures de scrap
    time.sleep(0.01) #On ajoute un petit délai pour ne pas surcharger le site de requêtes
    url_temp = 'https://www.imdb.com/title/'+ID+'/'
    response=session_obj.get(url_temp, headers={"User-Agent": "Mozilla/5.0"}) #On se fait passer pour une session normale ;) 
    html = response.content
    soup = bs(html, "html.parser")
    
    #Le bloc est composé de la partie de chaque page qui contient les informations utiles
    #On le transforme en dictionnaire
    bloc = soup.find("script", type="application/ld+json").string
    dictio = json.loads(bloc)
    
    #On retire dedans ce qui ne nous intéresse pas
    for inutile in superflu :
      dictio.pop(inutile, None)

    #on ajoute une ligne budget illico presto ATTENTION CA NE MARCHE PAS
    liste_budg = soup.find_all("label", class_="ipc-metadata-list-item__list-content-item")
    if len(liste_budg) >= 3 and '$' in liste_budg[2] :
      budget = liste_budg[2].string
      if budget == None :
        budget = "Non renseigné"
      else :
        budget = "".join([elemnt for elemnt in budget if elemnt.isdigit()])
      dictio['Budget'] = budget

    #L'allure du dictionnaire n'est pas parfaitement satisfaisante, par exemple chaque acteur est associé à une date de naissance,
    #à une photo, etc... On ne conserve que le nom des acteurs, et celui des réalisateurs
    
    if 'actor' in dictio :
      for acteur in dictio['actor'] :
        for inutile in superflu2 :
          acteur.pop(inutile, None)
      for indice, nom in enumerate(dictio['actor']) :
        dictio['actor'][indice] = nom['name']

    if 'director' in dictio :
      for directeur in dictio['director'] :
        for inutile in superflu2 :
          directeur.pop(inutile, None)
      for indice, nom in enumerate(dictio['director']) :
        dictio['director'][indice] = nom['name']
    
    if 'creator' in dictio :
      for createur in dictio['creator'] :
        createur.pop('@type', None)
      for indice, url in enumerate(dictio['creator']) :
        dictio['creator'][indice] = url['url']

    #Pour la société de production c'est un peu compliqué : on n'a qu'une URL
    #Ce qui n'est pas grave, puisqu'on peut retrouver son nom en scrapant cet url !
    #Mais le temps d'exécution explose si on le fait ; j'inclus donc ce code (complètement fonctionnel)
    #mais en pratique il prend trop de temps à tourner
    
    if 'creator' in dictio :
      for index, createur in enumerate(dictio['creator']) :
        url_temp = 'https://www.imdb.com'+createur
        response=session_obj.get(url_temp, headers={"User-Agent": "Mozilla/5.0"})
        html = response.content
        soup = bs(html, "lxml")
        compagnie_soup = soup.find("title")
        if compagnie_soup == None :
          compagnie = "Non renseigné"
        else :
          compagnie = compagnie_soup.string
        compagnie = compagnie[5:-40] #on garde que l'élément important du titre
        dictio['creator'][index] = compagnie

    #On ajoute au dictionnaire le pays d'origine, que l'on le trouve dans la date de sortie
    date_sortie_soup = soup.find("a", class_="ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link", href="/title/"+ID+"/releaseinfo?ref_=tt_dt_rdat")
    if date_sortie_soup == None :
      date_sortieV2 = "Non renseigné"
      pays = "Non renseigné"
    else :
      date_sortieV2 = date_sortie_soup.string
      b1 = date_sortieV2.find('(')
      b2 = date_sortieV2.find(')')
      pays = date_sortieV2[b1:b2]
      pays=pays[1:]
    dictio['Origine'] = pays

    #On ajoute dans le dataframe la ligne qui correspond au film
    df = df.append(dictio, ignore_index=True)
  except:
    print('Erreur au rang : '+str(compteur))

**Et voilà : nous avons construit une base de donnée grâce au scraping**

Nous ne conseillons pas de lancer ce code, puisqu'il nous a fallu plus d'une journée pour obtenir le dataframe, sans compter toutes les fois où notre ordinateur a eu des problèmes de connexion et interrompu l'exécution (ce qui nous a coûté au total 3 jours). En revanche, il est possible de le lancer sur quelques valeurs, pour obtenir un échantillon.

Nous mettrons à disposition le dataframe complet, pour pouvoir lancer le reste du code sans scraper à nouveau ces quelques 46.000 pages.

**La prochaine étape est donc de fusionner les dataframes et de transformer la classe des objets de chaque colonne, pour les rendre exploitables**

Le code suivant permet d'importer le dataframe du scrap (*df*) et de le fusionner avec le dataframe téléchargé et traité (*movies_rating_filtré*), en fabriquant une colonne commune pour permettre la jointure. On prend également le soin de rendre les éléments des colonnes *actor* et *director* comme des listes. Ce code fonctionne malgré l'avertissement qu'il renvoie.

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/Jeremstar/Succes_de_films-IMDb/main/Database/IMDB_2000votes.csv', 
                 converters={"actor": lambda x: x.strip("[]").split(", "), 'director': lambda y : y.strip("[]").split(", ")}) 

movies_rating_filtré['url']='/title/'+movies_rating_filtré['ID']+'/'
df_fusionné = df.merge(movies_rating_filtré, on='url',how='left')

On ordonne ensuite les colonnes dans l'ordre que l'on souhaite, on supprime celles qui posent problème.

In [3]:
df_fusionné = df_fusionné.reindex(columns=['ID_y','name','alternateName','url','contentRating','datePublished','genre','actor','director','creator','Origine','Budget','duration','keywords','Note_moyenne','Nombre_de_votes','ID_x'])	
df_fusionné =df_fusionné.drop(['ID_x', 'Budget', 'alternateName', 'creator'],axis=1) #On drop creator, pour les questions de scrap évoquées plus tôt, et budget parce qu'il est 
df_fusionné.rename(columns={'ID_y':'ID'}, inplace= True)

Certaines chaînes de caractères ne sont pas lisibles (on a par exemple des codes "&apos;" au lieu de véritables apostrophes).
La fonction suivante permet de résoudre ces problèmes, à la fois dans les listes et les string.

In [4]:
def correcteur(colonne, old, new) :
    if type(df_fusionné[colonne][0]) == list :
        all_crews = []
        for crew in df_fusionné[colonne]:
            crew_corrigé = []
            if crew != [] :
                for individu in crew :
                    crew_corrigé.append(individu.replace(old, new))
            all_crews.append(crew_corrigé)
        df_fusionné[colonne] = all_crews
    else :
        df_fusionné[colonne]= df_fusionné[colonne].str.replace(old, new)

On applique donc cette fonction de correction à toutes les colonnes qui en ont besoin.

In [5]:
correcteur('actor', '&apos;', "'")
correcteur('director', '&apos;', "'")
correcteur('name', '&apos;', "'")

Nous aurons par la suite recours souvent à une fonction qui permet d'obtenir toutes les valeurs uniques que contient une colonne, qu'elle contienne des listes ou des éléments simples. Elle a beaucoup servi dans notre travail de construction, donc nous la présentons pour que la démarche soit claire.

In [6]:
from pandas.core.common import flatten

def valeurs_possibles (colonne) :
    list_nonflat = df_fusionné[colonne]
    flat_list = list(flatten(list_nonflat))
    liste_valeurs = list(set(flat_list))
    return liste_valeurs

Grâce à cette fonction donc, nous pouvons harmoniser plus facilement les classifications d'âge des films.

In [ ]:
valeurs_possibles('contentRating')

In [ ]:
correcteur('contentRating', 'Tous Public', 'tous publics')
correcteur('contentRating', 'Tous Publics', 'tous publics')
correcteur('contentRating', 'Tous public', 'tous publics')
correcteur('contentRating', 'Tous publics', 'tous publics')
correcteur('contentRating', 'Not Rated', 'Not rated') #Absence de certificat
correcteur('contentRating', 'Unrated', 'Not rated') #Idem
correcteur('contentRating', '-12', '12')
correcteur('contentRating', '10 avec avertissement', '10')
correcteur('contentRating', '12 avec avertissement', '12')
correcteur('contentRating', 'Passed', 'Approved') #Classification d'avant 1968
correcteur('contentRating', '14+', '14')
correcteur('contentRating', '(Banned)', 'Banned')
correcteur('contentRating', '-16', '16')
correcteur('contentRating', 'TV-14', '14')
correcteur('contentRating', '16 avec avertissement', '16')
correcteur('contentRating', '-10', '10')
correcteur('contentRating', 'TV-PG', 'Accord parental')
correcteur('contentRating', 'PG-13', 'Accord parental')
correcteur('contentRating', 'M/PG', 'Accord parental')
correcteur('contentRating', 'PG', 'Accord parental')
correcteur('contentRating', '0+', 'tous publics')
correcteur('contentRating', 'E1Tous Publics+', 'tous publics')
correcteur('contentRating', 'E1Tous Public+', 'tous publics')
correcteur('contentRating', 'TV-13', '13')
correcteur('contentRating', 'R', '18')
correcteur('contentRating', 'GP', 'Accord parental')
correcteur('contentRating', 'MA-17', '17')
correcteur('contentRating', 'X', '18')
correcteur('contentRating', 'TV-Y7-FV', '7')
correcteur('contentRating', 'TV-Y7', '7')
correcteur('contentRating', 'TV-G', 'Tous Public')
correcteur('contentRating', 'G', 'Tous Public')
correcteur('contentRating', 'Tous Publics', 'tous publics')
correcteur('contentRating', 'Tout public', 'tous publics')
correcteur('contentRating', '1Tous Public', 'tous publics')
correcteur('contentRating', 'Tous Public+', 'tous publics')
correcteur('contentRating', 'NC-17', '18')
correcteur('contentRating', 'TV-MA', '18')
correcteur('contentRating', 'TV-T', 'tous publics')
correcteur('contentRating', 'M', '18')
correcteur('contentRating', 'E', 'Éducatif')
correcteur('contentRating', 'T', 'tous publics')
correcteur('contentRating', 'K-A', 'Erreur')
correcteur('contentRating', 'AO', 'Erreur')
correcteur('contentRating', 'Open', 'Erreur')
correcteur('contentRating', 'tous publicsV-Y', 'tous publics')
correcteur('contentRating', 'Éducatif1tous publics+', 'tous publics')
correcteur('contentRating', 'tous publicss avec avertissement', 'tous publics')
correcteur('contentRating', '1tous publics', 'tous publics')
correcteur('contentRating', 'tous publicss', 'tous publics')

Cette procédure est fastidieuse mais il est difficile de faire autrement pour harmoniser.

On convertit à présent les mois, pour les numéroter de 1 à 12.

In [8]:
df_fusionné['month'] = pd.DatetimeIndex(df_fusionné['datePublished']).month

On convertit ensuite la durée du film, pour passer par exemple de "PT1H32M" au nombre de minutes du film. Par souci de lisibilité, on construit d'abord une fonction de recodage, que l'on applique ensuite à la colonne.

In [9]:
def recodage_duree(duree) :
    if type(duree) == str :
        duree = duree[2:]
        if 'H' not in duree :
            duree_corrigee = int(duree[-3:-1])
        elif 'M' not in duree :
            duree_corrigee = int(duree[-2:-1])*60
        else :
            heures = int(duree[0])*60
            duree = duree[2:]
            minutes = int(duree[-3:-1])
            duree_corrigee = heures + minutes
    else :
        duree_corrigee = "Non renseigné"
    return duree_corrigee

In [10]:
df_fusionné['duration']=df_fusionné['duration'].apply(lambda row : recodage_duree(row))

À présent, on transforme les dates avec un véritable format date. On fait le choix d'écarter les 700 films pour lesquels la date n'est pas renseignée, en considérant qu'ils sont minoritaires.

In [ ]:
index_with_nan = df_fusionné.index[df_fusionné.loc[:,'datePublished'].isnull()]
df_fusionné.drop(index_with_nan,0, inplace=True)
df_fusionné = df_fusionné.sort_values(by='datePublished')
df_fusionné = df_fusionné.reset_index(drop = True)

In [26]:
import datetime
df_fusionné['datePublished'] = df_fusionné['datePublished'].apply(lambda x : datetime.datetime.strptime(str(x), '%Y-%m-%d'))

Le jeu de donné est à présent suffisamment propre pour être exploitable ! Affichons ses premières lignes pour voir son allure.

In [ ]:
df_fusionné.head(30)

In [27]:
type(df_fusionné['datePublished'][])

pandas._libs.tslibs.timestamps.Timestamp

In [11]:
df2 = df_fusionné.copy()

In [28]:
df_fusionné

,ID,name,url,contentRating,datePublished,genre,actor,director,Origine,duration,keywords,Note_moyenne,Nombre_de_votes,month
0,tt2221420,Sallie Gardner at a Gallop,/title/tt2221420/,tous publics,1878-06-15,"['Documentary', 'Short']","['Gilbert Domm', 'Sallie Gardner']",['Eadweard Muybridge'],United States,1,"19th century,1870s,nature,horse,first of its kind",7.4,3095.0,6.0
1,tt0392728,Roundhay Garden Scene,/title/tt0392728/,Not rated,1888-10-14,"['Documentary', 'Short']","['Annie Hartley', 'Adolphe Le Prince', 'Joseph...",['Louis Aimé Augustin Le Prince'],United Kingdom,1,"place name in title,first of its kind,year 188...",7.3,6253.0,10.0
2,tt0000005,Blacksmith Scene,/title/tt0000005/,Not rated,1893-05-09,"['Short', 'Comedy']","['Charles Kayser', 'John Ott']",['William K.L. Dickson'],United States,1,"blacksmith,national film registry,beer,two wor...",6.2,2555.0,5.0
3,tt0000008,Edison Kinetoscopic Record of a Sneeze,/title/tt0000008/,NaN,1894-01-09,"['Documentary', 'Short']",['Fred Ott'],['William K.L. Dickson'],United States,1,"national film registry,year 1894,1890s,19th ce...",5.4,2070.0,1.0
4,tt0177707,Dickson Experimental Sound Film,/title/tt0177707/,Not rated,1894-08-31,"['Short', 'Music']",['William K.L. Dickson'],['William K.L. Dickson'],United States,22,"same sex dance partners,gay interest,national ...",6.7,2378.0,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46076,tt14208870,The Fabelmans,/title/tt14208870/,Accord parental,2023-01-25,['Drama'],"['Michelle Williams', 'Gabriel LaBelle', 'Paul...",['Steven Spielberg'],France,151,"arizona,coming of age,childhood,father son rel...",8.1,8103.0,1.0
46077,tt14444726,Tár,/title/tt14444726/,18,2023-02-22,"['Drama', 'Music']","['Cate Blanchett', 'Noémie Merlant', 'Nina Hoss']",['Todd Field'],France,158,"female rear nudity,classical music,character n...",7.8,11155.0,2.0
46078,tt13833688,The Whale,/title/tt13833688/,18,2023-03-08,['Drama'],"['Brendan Fraser', 'Sadie Sink', 'Ty Simpkins']",['Darren Aronofsky'],France,117,"screenplay adapted by author,overweight man,ga...",8.3,5035.0,3.0
46079,tt4960748,Till,/title/tt4960748/,Accord parental,2023-03-22,"['Biography', 'Drama', 'History']","['Danielle Deadwyler', 'Jalyn Hall', 'Frankie ...",['Chinonye Chukwu'],France,130,"year 1955,based on real people,1950s,justice,m...",7.2,3851.0,3.0
